# Spelling correction

You receive the following files, available via SurfDrive:
- sentences_with_typos.txt: a file with two tab-separated columns, the first containing a numerical ID and the second containing a sentence in English
- SUBTLEXus.txt: a file with several tab-separated columns, containing data from the SUBTLEXus lexicon, a list of words derived from a part of the SUBTLEXus corpus containing movie subtitles with their frequency counts (and other pieces of information)
- LM_trainingCorpus.json: a file containing a pre-processed corpus in the form of a list of lists of strings, to be used to train a word-level statistical language model

You should carry out the following tasks:

- task1: comment the code to estimate the LM by adding doc-strings. 4 points available, you lose 1 point for every missing docstring. There are 6 docstrings to write, so writing 2 give you the same points as writing none: the rationale behind the grading approach is that writing only 2 is not a sufficient demonstration you understand the code.


- task2a: After reading in the files as pandas dataframes (mind how you import default strings, some of them may turn into NAs!), find which token contains a typo in the given sentences, where having a typo means that the word does not feature in SUBTLEXus - remember to tokenise the input sentences! There is one and only one word containing a typo in each sentence, as defined in this way: the typo can result from the insertion/deletion/substitution of one or two characters. You should submit a .json file containing a simple dictionary mapping the sentence ID (as an integer) to the mistyped word (as a string). 5 points available in total: for every incorrect word retrieved, you lose 0.5 point. If you retrieve 10 wrong mistyped words, you get no points even if the total number of mistyped words is higher than 10. If you submit a wrongly formatted file, you will get no points.


- task2b: read the target sentences manually. Some of them contain another mistyped word than the one you found in task2a, but were not detected because the typo resulted in a word which appears in SUBTLEXus. Discuss how you could automatically spot those mistyped words too using CL methods and resources: what information would you need? How would you use it? Reply in no more than 150 words. 3 points available, awarded based on how sensible the answer is.


- task3: find the words from SUBTLEXus with the smallest edit distance from each mistyped target (do not lowercase anything). You should return the 3 words at the smallest edit distance, sorted by edit distance. However, if there are more words at the same edit distance than the third closest, you should include all the words at the same edit distance. Therefore, supposing that the string 'abcdef' has two neighbors at edit distance 1, and four neighbors at edit distance 2, the third closest neighbor would be at edit distance 2, but there would be other three words at the same distance and you should thus return six neighbors for the target string. 5 points available: you loose 0.5 points for every wrong list of nearest neighbors you retrieve for a mistyped word (wrong means any of wrong words, wrong order, wrong edit distances, wrong data type). Submit a .json file containing a dictionary mapping sentence IDs (as integers) to lists of tuples, where each tuple contains first the word (as a string) and then the edit distance (as an integer), with tuples sorted by edit distance in ascending order (smallest edit distance first). The dictionary should have the following form (if you submit a wrongly formatted file, you will get no points):
    {id1: [(w1, 2), (w2, 2), (w3, 2)];
     id2: [(w6, 1), (w7, 1), (w8, 2), (w9, 2), (w10, 2)];
     ...}
     
     
- task4: use the list of candidate replacements you found in task3 to find the best one according to candidate frequency (derived from SUBTLEXus) - if two or more candidates have the exact same frequency in SUBTLEX, choose the one with the best edit distance. If two or more candidates at the same frequency also have the same edit distance, pick the one which comes first in alphabetical order. You should return a .json file containing a simple dictionary mapping the sentence ID (as an integer) to a tuple containing the best candidate replacement (as a string) and its SUBTLEXus frequency value (as an integer). 5 points available, you lose 1 point for each wrong best candidate:frequency pair you retrieve (if the candidate is right but the frequency doesn't match, you loose half a point). Not all candidates might appear in the LM training corpus: don't map to the UNK string though, or the perplexity estimate would not reflect the specific candidate, you can directly exclude them from the perplexity computation.


- task5: use the list of candidate replacements you found in task3 to find the best one according to its perplexity under a statistical language model of order 3 implemented using a Markov Chain with add-k smoothing (k=0.01) and estimated using the given corpus. If two or more candidates have the exact same perplexity in the input sentence, choose the one with the best edit distance. If two or more candidates at the same perplexity also have the same edit distance, follow alphabetical order. You should return a .json file containing a simple dictionary mapping the sentence ID (as an integer) to a tuple containing the best candidate replacement (as a string) and its perplexity under the specified language model (as a float). 5 points available, you lose 1 point for each wrong best candidate:frequency pair you retrieve (if the candidate is right but the perplexity doesn't match, you loose half a point).


- task6: Compare the candidate replacements found when considering frequency and when considering perplexity. Which are better? Do they match what you consider to be the right replacement? What extra resources/information would you use to pick better candidate replacements? 3 points available, awarded based on how sensible the answer is.


Name files as follows:
task[n]_group[m]_solution.json

such that the solutions from group 45 to task 2a should be contained in the file task2a_group45_solution.json, in the format specified in the task.

## Task1

In the cell below you find code to run a statistical language model, add the docstrings (you already find a blueprint) to complete the first task.

In [12]:
import operator
import numpy as np
from collections import defaultdict

class LM(object):
    
    def __init__(self, corpus, ngram_size=2, bos='+', eos='#', k=1):
        
        """
        :param corpus:      The collection of sentences to be used in the LM
        :param ngram_size:  The size of word combinations to be created while preprocessing
        :param k:           The fixed size to be added to avoid zero values (smoothing)
        :param bos:         The symbol for the beginning of a sentence
        :param eos:         The symbol for the end of a sentence
        
        This class creates an LM object with specific attributes 
        """
        
        self.k = k
        self.ngram_size = ngram_size
        self.bos = bos
        self.eos = eos
        self.corpus = corpus
        self.vocab = self.get_vocab()
        self.vocab.add(self.eos)
        self.vocab_size = len(self.vocab)
        
    def get_vocab(self):
        
        """
        :return: Returns the vocabulary, also in other words getting the different types.
        
        This method derives the vocabulary of our corpus
        """
        
        vocab = set()
        for sentence in self.corpus:
            for element in sentence:
                vocab.add(element)
        
        return vocab
                    
    def update_counts(self):
        
        """
        :return: Does not return but updates the count global variable
        
        This method creates a dictionary for the counts of specific words given the context of the preciding n_gram parts.
        """
        
        r = self.ngram_size - 1
        
        self.counts = defaultdict(dict)
        
        for sentence in self.corpus:
            s = [self.bos]*r + sentence + [self.eos]
            
            for idx in range(self.ngram_size-1, len(s)):
                ngram = self.get_ngram(s, idx)
                
                try:
                    self.counts[ngram[0]][ngram[1]] += 1
                except KeyError:
                    self.counts[ngram[0]][ngram[1]] = 1
                        
    def get_ngram(self, s, i):
        
        """
        :param s:   We get the sentence as input
        :param i:   We get the index as input
        :return:    Returns the the first parts of the n_gram and the last piece of the n_gram as target
        
        This method returns the n_gram having it's last word at the specificed index
        """
        
        ngram = s[i-(self.ngram_size-1):i+1]
        history = tuple(ngram[:-1])
        target = ngram[-1]
        return (history, target)
    
    def get_ngram_probability(self, history, target):
        
        """
        :param history: The preceeding words of our n_gram
        :param target:  The current/last word of our n_gram
        :return:        Returns the probability of the target n_gram
        
        This method calculates our probability of the target word given the history of preceding words in our ngram
        """
        
        try:
            ngram_tot = np.sum(list(self.counts[history].values())) + (self.vocab_size*self.k)
            try:
                transition_count = self.counts[history][target] + self.k
            except KeyError:
                transition_count = self.k
        except KeyError:
            transition_count = self.k
            ngram_tot = self.vocab_size*self.k
        
        return transition_count/ngram_tot 
    
    def perplexity(self, sentence):
        
        """
        :param string: the test sentence based on unseen data
        :return: the perplexity (calculated value) on how our model performs
        
        This method computes the perplexity, thus a value showing the intrinsic evaluation of our model
        """
        
        r = self.ngram_size - 1
        s = [self.bos]*r + sentence + [self.eos]
        

        probs = []
        for idx in range(self.ngram_size-1, len(s)):
            ngram = self.get_ngram(s, idx)
            probs.append(self.get_ngram_probability(ngram[0], ngram[1]))
                    
        entropy = np.log2(probs)
        avg_entropy = -1 * (sum(entropy) / len(entropy))
        return pow(2.0, avg_entropy)

In [13]:
import json
import nltk
import spacy
import pandas as pd
from collections import defaultdict

## Task2

In [14]:
# load the data
df_with_typos = pd.read_csv("sentence_with_typos.txt", sep="\t")
df_sublexus = pd.read_csv("SUBTLEXus.txt", sep="\t")

C:\Users\friez\AppData\Local\Temp\ipykernel_10184\415206871.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sublexus = pd.read_csv("SUBTLEXus.txt", sep="\t")


## Task 2a

In [15]:
# find mistyped words in the input sentences
result_dict2 = {}

#keepNan default
for i in range(df_with_typos.shape[0]):
    row = df_with_typos.loc[df_with_typos['ID'] == i+1, 'sentence'].item()
    tokens = nltk.word_tokenize(row)
    for token in tokens:
        if token in "?.,!":
            continue
        if token != "" and token not in df_sublexus["Word"].values:
            result_dict2[i+1] = token

print(result_dict2)
    
    
with open("task2a_ChristopheFriezasGoncalves_solution.json", "w") as outfile:
    json.dump(result_dict2, outfile)

{1: 'knoq', 2: 'reserachers', 3: 'grozn', 4: 'quolg', 5: 'waies', 6: 'wintr', 7: 'munors', 8: 'surgicaly', 9: 'aquire', 10: 'acomodate', 11: 'dats', 12: 'collegue', 13: 'layed', 14: 'cate', 15: 'giambic'}


## Task2b

In the first two sentences we have red and fiend which given the context of the sentences are also typo words yet they by themselves are correct words. One method would be a language model trained on a bigger corpus with sentences without typos to grasp the probabilities of words coming one after another. This model then may catch a word like red or fiend in this smaller corpus given that their probability of appearing inthe sentence after their specific sequence of world seems very very low thus marking them as typo.

## Task 3

In [16]:
# retrieve the nearest neighbors based on edit distance
words = df_sublexus["Word"].values
dict_res = {}
def set_dictEntry(idx, neighbors):
    count=1
    dict_res[idx]=[neighbors[0]]
    for i in range(1,len(neighbors)-1):
        if count >= 3 and neighbors[i-1][1]!=neighbors[i][1]:
            break
        count+=1
        dict_res[idx].append(neighbors[i])
        



for i, typo in result_dict2.items():
    target = typo
    min_edit = 100  # just a very high number
    neighbors = []
    for word in words:
        word = str(word)
        if word != target:
            d = nltk.edit_distance(word, target)
            
            if d>3:
                continue
            else:
                neighbors.append((word,d))
                neighbors.sort(key=lambda t: t[1])
    
    
    set_dictEntry(i,neighbors)

with open("task3_ChristopheFriezasGoncalves_solution.json", "w") as outfile:
    json.dump(dict_res, outfile)

In [17]:
print(dict_res)

{1: [('know', 1), ('knot', 1), ('knob', 1)], 2: [('researchers', 2), ('researcher', 3), ('researches', 3)], 3: [('grown', 1), ('groin', 1), ('groan', 1)], 4: [('quote', 2), ('quota', 2), ('quo', 2), ('quilt', 2), ('quell', 2), ('qualm', 2), ('quila', 2), ('quoad', 2), ('quoit', 2), ('quos', 2)], 5: [('waves', 1), ('wakes', 1), ('waits', 1), ('wages', 1), ('wails', 1), ('wares', 1), ('waxes', 1), ('wades', 1), ('waives', 1), ('waifs', 1), ('wanes', 1)], 6: [('winter', 1), ('wintry', 1), ('with', 2), ('want', 2), ('into', 2), ('went', 2), ('wants', 2), ('win', 2), ('wind', 2), ('wine', 2), ('winner', 2), ('wins', 2), ('wings', 2), ('wing', 2), ('minor', 2), ('hint', 2), ('diner', 2), ('winds', 2), ('ninth', 2), ('wit', 2), ('mint', 2), ('witty', 2), ('wits', 2), ('wiser', 2), ('pint', 2), ('wink', 2), ('finer', 2), ('wider', 2), ('windy', 2), ('intro', 2), ('hints', 2), ('wiener', 2), ('mints', 2), ('wilt', 2), ('wines', 2), ('miner', 2), ('liner', 2), ('pints', 2), ('lint', 2), ('winch'

## Task4: frequency

In [18]:
# pick the best candidate according to SUBTLEXus frequency counts
freq = dict()
words=df_sublexus["Word"].values
count=df_sublexus["FREQcount"].values
dict_res4 = {}
for i in range(len(words)):
    freq[words[i]] = count[i]
    

for i in range(1,16):
    candid = dict_res[i]
    best = []
    
    for j in candid:
        word = j[0]
        wordCom = (word,freq[word], j[1])
        best.append(wordCom)
    
    best.sort(key=lambda x: x[1], reverse=True)
    
    
    if best[0][1] == best[1][1]:
        nw = [x for x in best if x[1] == best[0][1]] 
        nw.sort(key=lambda x: x[2])
        
        if nw[0][2] == nw[0][2]:
            nw = [x for x in best if x[2] == nw[0][2] ]
            nw.sort(key=lambda x: x[0])
        best = nw
    
    dict_res4[i] = tuple((best[0][0],int(best[0][1])))
    print(result_dict2[i],  dict_res4[i])


with open("task4_ChristopheFriezasGoncalves_solution.json", "w") as outfile:
    json.dump(dict_res4, outfile)

knoq ('know', 291780)
reserachers ('researchers', 57)
grozn ('grown', 1275)
quolg ('quote', 488)
waies ('waves', 674)
wintr ('with', 257465)
munors ('minor', 654)
surgicaly ('strictly', 548)
aquire ('Squire', 157)
acomodate ('accommodate', 109)
dats ('days', 15592)
collegue ('college', 4344)
layed ('played', 2870)
cate ('care', 24748)
giambic ('gambit', 19)


## Task5: perplexity

In [19]:
# pick the best candidate according to perplexity under the given language model
import json

with open("LM_trainingCorpus.json") as f:
    data = json.load(f)

In [20]:
from collections import Counter, defaultdict

class Corpus(object):
    def __init__(self, n, corpus=None):
                
        self.sentences = corpus
        self.ngram_size = n
        
        self.frequencies = self.freq_distr()
        #self.filter_words()
        self.add_bos_eos()
        
    def freq_distr(self):
        count = Counter()
        
        for i in range(len(self.sentences)):
            for word in self.sentences[i]:
                count[word] += 1
                
        return count
    
    def filter_words(self):
        for i in range(len(self.sentences)):
            cleaned = []
            for word in self.sentences[i]:
                if word == "" or word in ".!?,":
                    continue
                else:
                    cleaned.append(word)
                
            self.sentences[i] = cleaned
    
    def add_bos_eos(self):
        for i in range(len(self.sentences)):
            new = ['#bos#']*(self.ngram_size-1)
            new.extend(self.sentences[i])
            new.append('#eos#')
            self.sentences[i] = new
            
            

In [21]:
class LM(object):
    
    def __init__(self, n, smoother='Laplace', k=1, lambdas=None):
        
        # make sure that no fixed quantity is added to counts when the smoother is not Laplace
        self.k = k
        self.ngram_size = n
        self.smoother = smoother
        self.lambdas = lambdas if lambdas else {i+1: 1/n for i in range(n)}
        
    def get_ngram(self, sentence, i, n):
        
        if n == 1:
            return sentence[i]
        else:
            ngram = sentence[i-(n-1):i+1]
            history = tuple(ngram[:-1])
            target = ngram[-1]
            return (history, target)
        
                    
    def update_counts(self, corpus, n):
        
        if self.ngram_size != corpus.ngram_size:
            raise ValueError("The corpus was pre-processed considering an ngram size of {} while the "
                             "language model was created with an ngram size of {}. \n"
                             "Please choose the same ngram size for pre-processing the corpus and fitting "
                             "the model.".format(corpus.ngram_size, self.ngram_size))
        
        self.counts = defaultdict(dict)
        # if the interpolation smoother is selected, then estimate transition counts for all possible ngram_sizes
        # smaller than the given one, otherwise stick with the input ngram_size
        ngram_sizes = [n] if self.smoother != 'Interpolation' else range(1,n+1)
        for ngram_size in ngram_sizes:
            self.counts[ngram_size] = defaultdict(dict) if ngram_size > 1 else Counter()
        for sentence in corpus.sentences:
            for ngram_size in ngram_sizes:
                for idx in range(n-1, len(sentence)):
                    ngram = self.get_ngram(sentence, idx, ngram_size)
                    if ngram_size == 1:
                        self.counts[ngram_size][ngram] += 1
                    else:
                        # it's faster to try to do something and catch an exception than to use an if statement to 
                        # check whether a condition is met beforehand. The if is checked everytime, the exception 
                        # is only catched the first time, after that everything runs smoothly
                        try:
                            self.counts[ngram_size][ngram[0]][ngram[1]] += 1
                        except KeyError:
                            self.counts[ngram_size][ngram[0]][ngram[1]] = 1
        
        # first loop through the sentences in the corpus, than loop through each word in a sentence
        self.vocab = {word for sentence in corpus.sentences for word in sentence}
        self.vocab_size = len(self.vocab)
    
    
    def get_ngram_probability(self, history, target):
        try:
            ngram_tot = np.sum(list(self.counts[self.ngram_size][history].values())) + (self.vocab_size*self.k)
            try:
                transition_count = self.counts[self.ngram_size][history][target] + self.k
            except KeyError:
                transition_count = self.k
        except KeyError:
            transition_count = self.k
            ngram_tot = self.vocab_size*self.k
            
        return transition_count/ngram_tot 
    
    def perplexity(self, test_corpus):
        probs = []
        for sentence in test_corpus.sentences:
            for idx in range(self.ngram_size-1, len(sentence)):
                ngram = self.get_ngram(sentence, idx, self.ngram_size)
                probs.append(self.get_ngram_probability(ngram[0], ngram[1]))
                        
        entropy = np.log2(probs)

        # this assertion makes sure that valid probabilities are retrieved, whose log must be <= 0
        assert all(entropy <= 0)
        
        
        avg_entropy = -1 * (sum(entropy) / len(entropy))
        
        return pow(2.0, avg_entropy)

In [22]:
corpus = Corpus(4,data)
ngram_model = LM(4, k=0.01, lambdas=None)
ngram_model.update_counts(corpus, 4)
dict_res5 = {}
sentences = {}


for i in range(df_with_typos.shape[0]):
    row = df_with_typos.loc[df_with_typos['ID'] == i+1, 'sentence'].item()
    tokens = nltk.word_tokenize(row)
    sentences[i+1]=tokens



for i in range(1,16):
    candid = dict_res[i]
    best = []
    replace = result_dict2[i]
    sentence = sentences[i]
    index = sentence.index(replace)
    
    
    for j in candid:
        repSentence = sentence[:]
        word = j[0]
        repSentence[index] = word
        wordCom = (word, ngram_model.perplexity(Corpus(4,[repSentence])), j[1])
        best.append(wordCom)
    
    best.sort(key=lambda x: x[1])
    
    
    if best[0][1] == best[1][1]:
        nw = [x for x in best if x[1] == best[0][1]] 
        nw.sort(key=lambda x: x[2])
        
        if nw[0][2] == nw[0][2]:
            nw = [x for x in best if x[2] == nw[0][2] ]
            nw.sort(key=lambda x: x[0])
        best = nw
    
    dict_res5[i] = (best[0][0],best[0][1])
    

print(dict_res5)
with open("task5_ChristopheFriezasGoncalves_solution.json", "w") as outfile:
    json.dump(dict_res5, outfile)

{1: ('know', 6121.518964425194), 2: ('researchers', 44977.18970730794), 3: ('groan', 7770.120723574704), 4: ('qualm', 74762.30658062035), 5: ('wades', 6420.0778660793085), 6: ('winter', 14222.749639803715), 7: ('jurors', 44017.860939303704), 8: ('surgical', 66731.89255192831), 9: ('Squire', 33355.42808237516), 10: ('accomodate', 49093.49848525223), 11: ('Kats', 40273.413009451724), 12: ('colleague', 2727.071187173628), 13: ('Fayed', 39395.67041280543), 14: ('care', 21888.81399625804), 15: ('iambic', 71457.10634471412)}


## Task 6

As we see for the 15 sentences frequency based typo replacement gives 5 out of 15 correct answers and perplexity gives 8 out of 15 correct answers. The perplexity thus shows better results when considering tokens based on edit distances. Adding that frequency does not take any context into account, just the fact that it may be a word that occurs often. Now we do not have any background on the training corpus thus we may improve our Language model if we may use a corpus that is based more on our sentences and their context. Given that we are looking for specific words for typos, lemmatizing and normalizing other than taking capitalization away is not possible, yet we can still try and fine tune our model, try different ngram sizes or train the model itself with more of unknown words tags, thus words like Squire, Fayed as seen in task 5 won't show up as option, given that squire and Fayed are not very common words in everyday language.